In [6]:
# %pip install peft evaluate transformers Levenshtein ipywidgets
# %pip install protobuf==3.20.3

In [6]:
import os
os.environ["TRANSFORMERS_DISABLE_CHAT_TEMPLATES"] = "1"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
os.environ["TRANSFORMERS_NO_ADDITIONAL_CHAT_TEMPLATES"] = "1"

In [7]:
from datasets import load_dataset, load_from_disk
# from UQA.canine_utils import preprocess_uqa, lora_config, print_trainable_parameters, normalize_answer, exact_match_score, f1_score, edit_distance_score, gold_answer, decode_prediction
from transformers import CanineTokenizer
from peft import LoraConfig, TaskType, get_peft_model
import re
import string
from collections import Counter
import numpy as np
import Levenshtein

from transformers import TrainingArguments, Trainer, TrainerCallback
import json
from huggingface_hub import HfApi, notebook_login, whoami

In [9]:
# notebook_login()
# whoami()

In [10]:
uqa_dataset = load_dataset("uqa/UQA")
uqa_train = uqa_dataset["train"].shuffle(seed=42).select(range(40000))
uqa_val = uqa_dataset["validation"].shuffle(seed=42).select(range(10000))

README.md:   0%|          | 0.00/898 [00:00<?, ?B/s]

data/train-00000-of-00001-bac007e8ca7192(…):   0%|          | 0.00/30.2M [00:00<?, ?B/s]

data/validation-00000-of-00001-cf8a6960d(…):   0%|          | 0.00/2.92M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/124745 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/16824 [00:00<?, ? examples/s]

In [11]:
from transformers import CanineTokenizer, CanineForQuestionAnswering
import torch
model_name = 'google/canine-s'
device = torch.device("cuda" if torch.cuda.is_available() else "mps")

tokenizer = CanineTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=False)
model = CanineForQuestionAnswering.from_pretrained(model_name, trust_remote_code=False)

tokenizer_config.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/528M [00:00<?, ?B/s]

Some weights of CanineForQuestionAnswering were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---

## Updated preprocessors!

Previously, we tried to apply the same approach we used in TYDIQA on UQA, the problem was the preprocessors were aligning the answer spans in units of **byte-level spans** instead of **character-level spans**. The calculations were adding byte-level offsets to the answer lengths, and since Urdu characters may be quantified in multiple bytes, the model was being fed the wrong spans -> GIGO!

In [ ]:
MAX_SEQ_LENGTH = 384
DOC_STRIDE = 128 # Increased stride for better overlap coverage

def preprocess_uqa(examples, tokenizer, max_length=MAX_SEQ_LENGTH, doc_stride=DOC_STRIDE, model_obj=None, indices=None):
    # Handle tokenizer/model limits safely
    tokenizer_max = getattr(tokenizer, "model_max_length", max_length)
    model_max = getattr(model_obj.config, "max_position_embeddings", None) if model_obj is not None else None
    max_allowed = max_length
    if tokenizer_max is not None and tokenizer_max > 0:
        max_allowed = min(max_allowed, tokenizer_max)
    if model_max is not None and model_max > 0:
        max_allowed = min(max_allowed, model_max)

    questions = [q.strip() for q in examples["question"]]
    contexts = examples["context"]
    answers = examples["answer"]
    answer_starts = examples["answer_start"]
    
    encoded = {
        "input_ids": [], 
        "attention_mask": [], 
        "token_type_ids": [],
        "start_positions": [], 
        "end_positions": [], 
        "overflow_to_sample_mapping": []
    }

    for i, (question, context, answer, answer_start) in enumerate(zip(questions, contexts, answers, answer_starts)):
        example_idx = indices[i] if indices is not None else i
        
        # CANINE encodes to characters directly. 
        # add_special_tokens=False gives us raw character IDs.
        question_ids = tokenizer.encode(question, add_special_tokens=False)
        context_ids = tokenizer.encode(context, add_special_tokens=False)

        # 1. Setup Targets (DIRECT MAPPING)
        # UQA answer_start is a CHARACTER index. CANINE tokens are CHARACTERS.
        # Therefore, answer_start maps 1:1 to the context_ids index.
        if answer and answer_start != -1:
            gold_char_start = answer_start
            gold_char_end = answer_start + len(answer) # Points to char AFTER the answer
        else:
            gold_char_start = -1
            gold_char_end = -1

        # 2. Calculate Window Size
        # [CLS] + Question + [SEP] + Context + [SEP]
        special_tokens_count = tokenizer.num_special_tokens_to_add(pair=True)
        max_context_length = max_allowed - len(question_ids) - special_tokens_count
        
        if max_context_length <= 0:
            # Edge case: Question is too long, skip or truncate question (skipping here for safety)
            continue

        # 3. Sliding Window Loop
        stride_step = max_context_length - doc_stride
        if stride_step <= 0: stride_step = max_context_length # Fallback if doc_stride is too big
        
        for chunk_start_idx in range(0, len(context_ids), stride_step):
            chunk_end_idx = min(chunk_start_idx + max_context_length, len(context_ids))
            context_chunk = context_ids[chunk_start_idx:chunk_end_idx]
            
            # Build inputs using tokenizer utility to handle special tokens correctly
            input_ids = tokenizer.build_inputs_with_special_tokens(question_ids, context_chunk)
            token_type_ids = tokenizer.create_token_type_ids_from_sequences(question_ids, context_chunk)
            attention_mask = [1] * len(input_ids)
            
            # Calculate Offset: Where does the context actually start in input_ids?
            # Typically: [CLS] (1) + Q_Len + [SEP] (1) = Start of Context
            # We calculate this dynamically to be safe:
            sep_indices = [k for k, x in enumerate(input_ids) if x == tokenizer.sep_token_id]
            if not sep_indices: 
                continue # Should not happen
            context_offset_in_input = sep_indices[0] + 1 
            
            # 4. Label Assignment
            # Check if the answer lies ENTIRELY within this specific chunk
            is_answer_in_chunk = (
                gold_char_start >= chunk_start_idx and 
                gold_char_end <= chunk_end_idx and
                gold_char_start != -1
            )
            
            if is_answer_in_chunk:
                # Map global context index to local window index
                start_pos = context_offset_in_input + (gold_char_start - chunk_start_idx)
                # -1 because end_positions is usually inclusive in HF Trainers
                end_pos = context_offset_in_input + (gold_char_end - chunk_start_idx) - 1
            else:
                # Label as [CLS] (index 0) if answer is not here
                start_pos = 0
                end_pos = 0

            # 5. Padding
            pad_len = max_allowed - len(input_ids)
            if pad_len > 0:
                input_ids += [tokenizer.pad_token_id] * pad_len
                attention_mask += [0] * pad_len
                token_type_ids += [0] * pad_len
            
            # 6. Final Safety Truncation (just in case)
            if len(input_ids) > max_allowed:
                input_ids = input_ids[:max_allowed]
                attention_mask = attention_mask[:max_allowed]
                token_type_ids = token_type_ids[:max_allowed]
                # If labels were pushed out by truncation, reset to 0
                if start_pos >= max_allowed or end_pos >= max_allowed:
                    start_pos = 0
                    end_pos = 0

            encoded["input_ids"].append(input_ids)
            encoded["attention_mask"].append(attention_mask)
            encoded["token_type_ids"].append(token_type_ids)
            encoded["start_positions"].append(start_pos)
            encoded["end_positions"].append(end_pos)
            encoded["overflow_to_sample_mapping"].append(example_idx)
            
            # Break loop if this chunk reached the end of the context
            if chunk_end_idx >= len(context_ids):
                break

    return encoded


In [18]:
# LoRA config
lora_config = LoraConfig(
    task_type=TaskType.QUESTION_ANS,
    r=32,   # changed from 8
    lora_alpha=64,  # changed from 32
    lora_dropout=0.1,
    target_modules=["query", "value", "key"],   # added key, output.dense
    bias="none",
    modules_to_save=["qa_outputs"],
)

def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")


In [19]:
# preprocess the train and val splits
processed_train = uqa_train.map(lambda examples, indices: preprocess_uqa(examples, tokenizer, indices=indices), batched=True, remove_columns=uqa_train.column_names, with_indices=True)
processed_val = uqa_val.map(lambda examples, indices: preprocess_uqa(examples, tokenizer, indices=indices), batched=True, remove_columns=uqa_val.column_names, with_indices=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3179 > 2048). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [20]:
processed_train

Dataset({
    features: ['input_ids', 'attention_mask', 'token_type_ids', 'start_positions', 'end_positions', 'overflow_to_sample_mapping'],
    num_rows: 28930
})

In [21]:
processed_val

Dataset({
    features: ['input_ids', 'attention_mask', 'token_type_ids', 'start_positions', 'end_positions', 'overflow_to_sample_mapping'],
    num_rows: 7341
})

In [22]:
processed_train.save_to_disk("/kaggle/working/cache/processed_train_uqa")
processed_val.save_to_disk("/kaggle/working/cache/processed_val_uqa")   # cached it


processed_train = load_from_disk("/kaggle/working/cache/processed_train_uqa")
processed_val = load_from_disk("/kaggle/working/cache/processed_val_uqa")

Saving the dataset (0/1 shards):   0%|          | 0/28930 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7341 [00:00<?, ? examples/s]

In [23]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")



In [24]:
# build LoRA model

peft_model = get_peft_model(model, lora_config)
peft_model.gradient_checkpointing_enable()
print_trainable_parameters(peft_model)

trainable params: 2065922 || all params: 134150404 || trainable%: 1.5400043074040985


In [25]:
# evals


def normalize_answer(text):
    text = (text or "").lower()
    def remove_articles(s):
        return re.sub(r"\b(a|an|the)\b", " ", s)
    def remove_punctuation(s):
        return "".join(ch for ch in s if ch not in string.punctuation)
    def white_space_fix(s):
        return " ".join(s.split())
    return white_space_fix(remove_articles(remove_punctuation(text)))

def exact_match_score(prediction, ground_truth):
    return float(normalize_answer(prediction) == normalize_answer(ground_truth))

def f1_score(prediction, ground_truth):
    pred_tokens = normalize_answer(prediction).split()
    gold_tokens = normalize_answer(ground_truth).split()
    if not gold_tokens:
        return 1.0 if not pred_tokens else 0.0
    if not pred_tokens:
        return 0.0
    common = Counter(pred_tokens) & Counter(gold_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(gold_tokens)
    return 2 * precision * recall / (precision + recall)

def edit_distance_score(prediction, ground_truth):
    pred_norm = normalize_answer(prediction)
    gold_norm = normalize_answer(ground_truth)
    if not gold_norm and not pred_norm:
        return 1.0
    if not gold_norm or not pred_norm:
        return 0.0
    distance = Levenshtein.distance(pred_norm, gold_norm)
    max_len = max(len(pred_norm), len(gold_norm))
    return 1.0 - (distance / max_len) if max_len > 0 else 1.0

def gold_answer(example):
    # Extracts the gold answer substring from the context using character offsets
    answer = example.get("answer")
    context = example.get("context")
    answer_start = example.get("answer_start", -1)
    if answer and answer_start is not None and answer_start != -1:
        return context[answer_start: answer_start + len(answer)]
    return "[CLS]"


def decode_prediction(input_ids, start_idx, end_idx, tokenizer=None):
    if start_idx > end_idx:
        start_idx, end_idx = end_idx, start_idx
    if tokenizer is None:
        raise ValueError("Tokenizer must be provided for decoding.")
    cls_index = input_ids.index(tokenizer.cls_token_id)
    # If both point to CLS token, return [CLS] sentinel
    if start_idx == cls_index and end_idx == cls_index:
        return "[CLS]"
    start_idx = max(start_idx, 0)
    end_idx = min(end_idx, len(input_ids) - 1)
    if start_idx > end_idx:
        return "[CLS]"
    text = tokenizer.decode(input_ids[start_idx:end_idx + 1], skip_special_tokens=True)
    text = text.strip()
    return text if text else "[CLS]"


def evaluate_checkpoint(checkpoint_path=None, model_instance=None, eval_dataset=None):
    """Evaluate either a checkpoint path (loads model) or a provided model instance.

    - checkpoint_path: path to checkpoint folder
    - model_instance: an in-memory model (preferably a PeftModel or CanineForQuestionAnswering)
    - eval_dataset: optional dataset to evaluate; if None the default processed_val will be used
    """
    if eval_dataset is None:
        eval_dataset = processed_val

    # If a model_instance is given, use it directly (avoid re-loading a fresh base model)
    if model_instance is not None:
        eval_model = model_instance
    else:
        base_model = CanineForQuestionAnswering.from_pretrained(model_name, trust_remote_code=False)
        eval_model = get_peft_model(base_model, lora_config)
        # Try loading adapter weights; fall back to PeftModel.from_pretrained if needed
        try:
            eval_model.load_adapter(checkpoint_path)
        except Exception:
            from peft import PeftModel
            eval_model = PeftModel.from_pretrained(base_model, checkpoint_path)

    eval_model.to(device)

    eval_args = TrainingArguments(
        # Small evaluation config; uses cpu/mps if no gpu during eval
        output_dir="outputs/canine-s-uqa",
        per_device_eval_batch_size=16,
        dataloader_drop_last=False,
        fp16=True,
        bf16=False,
        report_to="none",
    )

    # Run evaluation via a lightweight Trainer so prediction loop is standard
    eval_trainer = Trainer(
        model=eval_model,
        args=eval_args,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )

    predictions = eval_trainer.predict(eval_dataset)
    start_logits, end_logits = predictions.predictions
    best_predictions = {}
    for feature_index, feature in enumerate(eval_dataset):
        sample_idx = int(feature["overflow_to_sample_mapping"])
        input_ids = feature["input_ids"]
        start_idx = int(np.argmax(start_logits[feature_index]))
        end_idx = int(np.argmax(end_logits[feature_index]))
        score = float(start_logits[feature_index][start_idx] + end_logits[feature_index][end_idx])
        prediction_text = decode_prediction(input_ids, start_idx, end_idx, tokenizer=tokenizer)
        stored = best_predictions.get(sample_idx)
        if stored is None or score > stored[0]:
            best_predictions[sample_idx] = (score, prediction_text)

    em_scores = []
    f1_scores = []
    edit_dist_scores = []
    for sample_idx, (_, prediction_text) in best_predictions.items():
        reference = gold_answer(uqa_val[int(sample_idx)])
        em_scores.append(exact_match_score(prediction_text, reference))
        f1_scores.append(f1_score(prediction_text, reference))
        edit_dist_scores.append(edit_distance_score(prediction_text, reference))

    em = float(np.mean(em_scores)) if em_scores else 0.0
    f1 = float(np.mean(f1_scores)) if f1_scores else 0.0
    edit_dist = float(np.mean(edit_dist_scores)) if edit_dist_scores else 0.0
    print(f"Examples evaluated: {len(em_scores)}")
    print(f"Exact Match: {em * 100:.2f}")
    print(f"F1: {f1 * 100:.2f}")
    print(f"Edit Distance (normalized): {edit_dist * 100:.2f}")
    return {"exact_match": em, "f1": f1, "edit_distance": edit_dist}


In [26]:
training_args = TrainingArguments(
    output_dir="outputs/canine-s-uqa",
    
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    
    num_train_epochs=1,
    learning_rate=3e-4,
    weight_decay=0.01,
    eval_strategy="no",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    logging_steps=25,
    fp16=True,
    bf16=False,
    report_to="none",
    push_to_hub=True,
    hub_model_id="VohraAK/canine-s-uqa",
    hub_strategy="checkpoint",
    )

class CustomEvalCallback(TrainerCallback):
    def __init__(self, eval_func, eval_dataset, use_in_memory_model=True, verbose=True):
        self.eval_func = eval_func
        self.eval_dataset = eval_dataset
        self.use_in_memory_model = use_in_memory_model
        self.verbose = verbose
        # trainer reference (set after trainer exists)
        self.trainer = None

    def on_save(self, args, state, control, model=None, **kwargs):
        checkpoint_path = f"{args.output_dir}/checkpoint-{state.global_step}"
        if self.verbose:
            print(f"\n🔍 Running custom evaluation at step {state.global_step}...")

        # Prefer evaluating the in-memory trainer model (fast + avoids re-loading)
        if self.use_in_memory_model and self.trainer is not None:
            if self.verbose:
                print("Using in-memory model for evaluation (no reloading).")
            try:
                metrics = self.eval_func(checkpoint_path=None, model_instance=self.trainer.model, eval_dataset=self.eval_dataset)
            except Exception as e:
                print("⚠️ in-memory evaluation failed, falling back to checkpoint load:", e)
                metrics = self.eval_func(checkpoint_path)
        else:
            metrics = self.eval_func(checkpoint_path)

        # record metrics in state.log_history
        state.log_history.append({
            "step": state.global_step,
            "eval_exact_match": metrics.get("exact_match"),
            "eval_f1": metrics.get("f1"),
            "eval_edit_distance": metrics.get("edit_distance"),
        })

        if self.verbose:
            print(f"✅ Step {state.global_step}: EM={metrics.get('exact_match',0)*100:.2f}, F1={metrics.get('f1',0)*100:.2f}, EditDist={metrics.get('edit_distance',0)*100:.2f}")

        # Update trainer_state.json to include custom metrics
        state_path = f"{checkpoint_path}/trainer_state.json"
        try:
            with open(state_path, 'r') as f:
                state_dict = json.load(f)
            state_dict['log_history'] = state.log_history
            with open(state_path, 'w') as f:
                json.dump(state_dict, f, indent=2)
            if self.verbose:
                print(f"💾 Updated trainer_state.json with custom metrics")
        except Exception as e:
            if self.verbose:
                print(f"⚠️  Warning: Could not update trainer_state.json: {e}")

        try:
            if self.verbose:
                print(f"☁️  Pushing checkpoint-{state.global_step} to Hub...")
            api = HfApi()
            api.upload_folder(
                folder_path=checkpoint_path,
                repo_id=args.hub_model_id,
                path_in_repo=f"checkpoint-{state.global_step}",
                commit_message=f"Add checkpoint {state.global_step} (EM={metrics.get('exact_match',0)*100:.1f}%, F1={metrics.get('f1',0)*100:.1f}%)",
                repo_type="model"
            )
            if self.verbose:
                print(f"✅ Pushed checkpoint-{state.global_step} to Hub")
        except Exception as e:
            if self.verbose:
                print(f"⚠️  Warning: Could not push to Hub: {e}")

        return control

In [22]:
trainer_cb = CustomEvalCallback(evaluate_checkpoint, processed_val, use_in_memory_model=True)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=processed_train,
    eval_dataset=processed_val,
    callbacks=[trainer_cb],
)


No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
25,5.919800
50,5.689400
75,5.488100
100,5.369100
125,5.171900
150,5.064900
175,4.856200
200,4.693500
225,4.596600
250,4.623400



🔍 Running custom evaluation at step 500...


Some weights of CanineForQuestionAnswering were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/tmp/ipykernel_117/733281207.py:107: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(
No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used inste

Examples evaluated: 1000
Exact Match: 32.90
F1: 32.90
Edit Distance (normalized): 33.03
✅ Step 500: EM=32.90, F1=32.90, EditDist=33.03
💾 Updated trainer_state.json with custom metrics
☁️  Pushing checkpoint-500 to Hub...
✅ Pushed checkpoint-500 to Hub


...

Some weights of CanineForQuestionAnswering were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/tmp/ipykernel_117/733281207.py:107: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(
No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used inste

Examples evaluated: 1000
Exact Match: 33.10
F1: 33.10
Edit Distance (normalized): 33.23
✅ Step 2000: EM=33.10, F1=33.10, EditDist=33.23
💾 Updated trainer_state.json with custom metrics
☁️  Pushing checkpoint-2000 to Hub...
✅ Pushed checkpoint-2000 to Hub


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



🔍 Running custom evaluation at step 2500...


Some weights of CanineForQuestionAnswering were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/tmp/ipykernel_117/733281207.py:107: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(
No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used inste

In [27]:
# Diagnostic cell (fixed): Investigate preprocessing and truncation for many samples
import random
from transformers import AutoTokenizer

try:
    tokenizer = AutoTokenizer.from_pretrained("google/canine-s")
except Exception:
    tokenizer = None

num_samples = 20  # Number of samples to check
results = []
for split_name, orig_data, proc_data in [
    ("train", uqa_train, processed_train),
    ("val", uqa_val, processed_val)
]:
    indices = random.sample(range(len(proc_data)), min(num_samples, len(proc_data)))
    for idx in indices:
        proc = proc_data[idx]
        # Use overflow_to_sample_mapping to get the correct original index
        orig_idx = proc["overflow_to_sample_mapping"]
        orig = orig_data[orig_idx]
        input_ids = proc["input_ids"]
        start_pos = proc["start_positions"]
        end_pos = proc["end_positions"]
        gold_answer = orig.get("gold_answer", orig.get("answer", ""))
        context = orig.get("context", "")
        question = orig.get("question", "")
        # Decode input_ids to text
        if tokenizer:
            decoded_text = tokenizer.decode(input_ids, skip_special_tokens=False)
        else:
            decoded_text = str(input_ids)
        # Extract predicted answer span
        if 0 <= start_pos < len(input_ids) and 0 <= end_pos < len(input_ids):
            pred_span = tokenizer.decode(input_ids[start_pos:end_pos+1], skip_special_tokens=False) if tokenizer else str(input_ids[start_pos:end_pos+1])
        else:
            pred_span = "[CLS or invalid]"
        # Check if gold answer is present in decoded text
        gold_in_decoded = gold_answer in decoded_text
        # Check if pred_span matches gold answer
        pred_matches_gold = pred_span.strip() == gold_answer.strip()
        results.append({
            "split": split_name,
            "proc_idx": idx,
            "orig_idx": orig_idx,
            "question": question,
            "context_snippet": context[:120],
            "gold_answer": gold_answer,
            "decoded_text_snippet": decoded_text[:120],
            "start_pos": start_pos,
            "end_pos": end_pos,
            "pred_span": pred_span,
            "gold_in_decoded": gold_in_decoded,
            "pred_matches_gold": pred_matches_gold
        })

import pandas as pd
results_df = pd.DataFrame(results)
problem_cases = results_df[(~results_df["gold_in_decoded"]) | (~results_df["pred_matches_gold"])]
display(problem_cases)
display(results_df.head(10))

,split,proc_idx,orig_idx,question,context_snippet,gold_answer,decoded_text_snippet,start_pos,end_pos,pred_span,gold_in_decoded,pred_matches_gold
0,train,28470,39386,ایک مرکب نظام کا استعمال کیا کرتا ہے؟,تحریری جاپانی میں کانا کے نام سے جانا جانے وال...,جدید جاپانی تحریر,ایک مرکب نظام کا استعمال کیا کرتا ہے؟تحریری ...,316,325,نے آسانی س,False,False
1,train,11705,16189,ایچ آئی وی کی جینوٹائپک شناخت نے بعد میں کیا م...,عام طور پر ایک متعدی ایجنٹ کی مخصوص شناخت کے ل...,جغرافیائی اصل,ایچ آئی وی کی جینوٹائپک شناخت نے بعد میں کیا ...,66,73,ں ایچ آئ,False,False
2,train,11321,15651,کون سی ویڈیوز اکثر براہ راست یوٹیوب سے ڈاؤن لو...,یوٹیوب عام طور پر اپنی ویڈیوز کے لئے ڈاؤن لوڈ ...,صدر باراک اوباما کے ہفتہ وار پتوں,کون سی ویڈیوز اکثر براہ راست یوٹیوب سے ڈاؤن ل...,167,184,رفیس کے ذریعے دیکھ,True,False
3,train,7244,10107,تھورنگیا کی آبادی کا کتنا حصہ کیتھولک ہے؟,پروٹسٹنٹ ریفارمیشن کے بعد سے ، تھورنگیا میں سب...,آبادی کا 7.8٪,تھورنگیا کی آبادی کا کتنا حصہ کیتھولک ہے؟پرو...,260,267,زیادہ آب,False,False
4,train,21081,29278,خط کے دو وصول کنندگان کو کس چیز پر توجہ دینے ک...,2015 میں بیونسے نے ایک کھلے خط پر دستخط کیے جس...,خواتین,خط کے دو وصول کنندگان کو کس چیز پر توجہ دینے ...,165,168,نکوس,True,False
5,train,6729,9402,قیامت کے بارے میں ایک مومن کو عام طور پر کیا ا...,عیسائیوں کا خیال ہے کہ یسوع کی موت انسانیت کو ...,خوشی,قیامت کے بارے میں ایک مومن کو عام طور پر کیا ...,193,195,مت,True,False
7,train,24292,33685,کتنی حاملہ خواتین حمل کے نتیجے میں مر جاتی ہیں؟,گنی بساو کے لئے 2010 کی زچگی کی شرح اموات فی 1...,آٹھہ میں سے ایک,کتنی حاملہ خواتین حمل کے نتیجے میں مر جاتی ہی...,246,255,ر 5 سال سے,True,False
8,train,22041,30575,کس قسم کی کارکردگی کے لئے کسی بھی دیئے گئے موس...,کلاسیکل میوزک کے ریپرٹوری کی کارکردگی کے لئے ن...,کلاسیکل میوزک,کس قسم کی کارکردگی کے لئے کسی بھی دیئے گئے مو...,94,100,کلاسیکل,True,False
9,train,17110,23843,آئل آف مین پر الزبتھ کا عنوان کیا ہے؟,الزبتھ نے پورے کامن ویلتھ میں بہت سے عنوانات ا...,لارڈ آف مین,آئل آف مین پر الزبتھ کا عنوان کیا ہے؟ہ اور ج...,47,52,یکا می,True,False
10,train,4212,5905,پوسٹ پنک کی مردہ راکھ سے کیا شروعاتیں اٹھیں؟,پوسٹ پنک ایک ایککلیکٹک صنف تھی جس کے نتیجے میں...,مختلف بعد کی صنفوں,پوسٹ پنک کی مردہ راکھ سے کیا شروعاتیں اٹھیں؟...,139,149,ے سفید اور,True,False


,split,proc_idx,orig_idx,question,context_snippet,gold_answer,decoded_text_snippet,start_pos,end_pos,pred_span,gold_in_decoded,pred_matches_gold
0,train,28470,39386,ایک مرکب نظام کا استعمال کیا کرتا ہے؟,تحریری جاپانی میں کانا کے نام سے جانا جانے وال...,جدید جاپانی تحریر,ایک مرکب نظام کا استعمال کیا کرتا ہے؟تحریری ...,316,325,نے آسانی س,False,False
1,train,11705,16189,ایچ آئی وی کی جینوٹائپک شناخت نے بعد میں کیا م...,عام طور پر ایک متعدی ایجنٹ کی مخصوص شناخت کے ل...,جغرافیائی اصل,ایچ آئی وی کی جینوٹائپک شناخت نے بعد میں کیا ...,66,73,ں ایچ آئ,False,False
2,train,11321,15651,کون سی ویڈیوز اکثر براہ راست یوٹیوب سے ڈاؤن لو...,یوٹیوب عام طور پر اپنی ویڈیوز کے لئے ڈاؤن لوڈ ...,صدر باراک اوباما کے ہفتہ وار پتوں,کون سی ویڈیوز اکثر براہ راست یوٹیوب سے ڈاؤن ل...,167,184,رفیس کے ذریعے دیکھ,True,False
3,train,7244,10107,تھورنگیا کی آبادی کا کتنا حصہ کیتھولک ہے؟,پروٹسٹنٹ ریفارمیشن کے بعد سے ، تھورنگیا میں سب...,آبادی کا 7.8٪,تھورنگیا کی آبادی کا کتنا حصہ کیتھولک ہے؟پرو...,260,267,زیادہ آب,False,False
4,train,21081,29278,خط کے دو وصول کنندگان کو کس چیز پر توجہ دینے ک...,2015 میں بیونسے نے ایک کھلے خط پر دستخط کیے جس...,خواتین,خط کے دو وصول کنندگان کو کس چیز پر توجہ دینے ...,165,168,نکوس,True,False
5,train,6729,9402,قیامت کے بارے میں ایک مومن کو عام طور پر کیا ا...,عیسائیوں کا خیال ہے کہ یسوع کی موت انسانیت کو ...,خوشی,قیامت کے بارے میں ایک مومن کو عام طور پر کیا ...,193,195,مت,True,False
6,train,21684,30082,کس سال چرچ کی طرف سے کینن قانون کے لئے ایک نیا...,1983 میں ، چرچ نے کینن قانون کا ایک نیا کوڈ جا...,1983,کس سال چرچ کی طرف سے کینن قانون کے لئے ایک نی...,70,73,1983,True,True
7,train,24292,33685,کتنی حاملہ خواتین حمل کے نتیجے میں مر جاتی ہیں؟,گنی بساو کے لئے 2010 کی زچگی کی شرح اموات فی 1...,آٹھہ میں سے ایک,کتنی حاملہ خواتین حمل کے نتیجے میں مر جاتی ہی...,246,255,ر 5 سال سے,True,False
8,train,22041,30575,کس قسم کی کارکردگی کے لئے کسی بھی دیئے گئے موس...,کلاسیکل میوزک کے ریپرٹوری کی کارکردگی کے لئے ن...,کلاسیکل میوزک,کس قسم کی کارکردگی کے لئے کسی بھی دیئے گئے مو...,94,100,کلاسیکل,True,False
9,train,17110,23843,آئل آف مین پر الزبتھ کا عنوان کیا ہے؟,الزبتھ نے پورے کامن ویلتھ میں بہت سے عنوانات ا...,لارڈ آف مین,آئل آف مین پر الزبتھ کا عنوان کیا ہے؟ہ اور ج...,47,52,یکا می,True,False


In [28]:
# Accuracy: fraction of rows where pred_span matches gold_answer (exact string match)
accuracy = (results_df["pred_matches_gold"]).mean()

# Precision: among rows where pred_span is non-empty, fraction that matches gold_answer
non_empty_pred = results_df["pred_span"].str.strip() != ""
precision = (results_df["pred_matches_gold"] & non_empty_pred).sum() / non_empty_pred.sum()

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")

Accuracy: 0.025
Precision: 0.025


In [29]:
# Print first 10 problematic cases for inspection
for i, row in results_df[(~results_df["gold_in_decoded"]) | (~results_df["pred_matches_gold"])].head(10).iterrows():
    print(f"Case {i+1}:")
    print(f"  Split: {row['split']}")
    print(f"  Question: {row['question']}")
    print(f"  Context snippet: {row['context_snippet']}")
    print(f"  Gold answer: {row['gold_answer']}")
    print(f"  Decoded input snippet: {row['decoded_text_snippet']}")
    print(f"  Start pos: {row['start_pos']}, End pos: {row['end_pos']}")
    print(f"  Predicted span: {row['pred_span']}")
    print(f"  Gold in decoded: {row['gold_in_decoded']}")
    print(f"  Pred matches gold: {row['pred_matches_gold']}")
    print("-" * 80)

Case 1:
  Split: train
  Question: ایک مرکب نظام کا استعمال کیا کرتا ہے؟
  Context snippet: تحریری جاپانی میں کانا کے نام سے جانا جانے والا ایک جوڑا حروف بھی شامل ہے ، جو جاپانی کے حروف کی نمائندگی کرنے کے لئے من
  Gold answer: جدید جاپانی تحریر
  Decoded input snippet: ایک مرکب نظام کا استعمال کیا کرتا ہے؟تحریری جاپانی میں کانا کے نام سے جانا جانے والا ایک جوڑا حروف بھی شامل ہے ، جو جا
  Start pos: 316, End pos: 325
  Predicted span: نے آسانی س
  Gold in decoded: False
  Pred matches gold: False
--------------------------------------------------------------------------------
Case 2:
  Split: train
  Question: ایچ آئی وی کی جینوٹائپک شناخت نے بعد میں کیا ممکن بنایا؟
  Context snippet: عام طور پر ایک متعدی ایجنٹ کی مخصوص شناخت کے لئے ایک اشارہ صرف اس وقت ہوتا ہے جب اس طرح کی شناخت بیماری کے علاج یا روک ت
  Gold answer: جغرافیائی اصل
  Decoded input snippet: ایچ آئی وی کی جینوٹائپک شناخت نے بعد میں کیا ممکن بنایا؟ونٹیز میں ایچ آئی وی کی ظاہری شکل کے بارے میں ان مطالعات نے وا
  Start 